<a href="https://colab.research.google.com/github/zhangzixuan1997/Python/blob/master/Kfold_with_Keras_Student%20Performance%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We got a RMSE of 40 for the example of Student Performance.
[Student Performace_Sean Zhang](https://https://github.com/zhangzixuan1997/Python/blob/master/Student_Performace-A%20simple%20deep%20learning%20early%20stopping%20example.ipynb)
What about we refine the model with k fold more specifications in the NN hyperparameters? 

In [7]:
import pandas as pd
from sklearn import metrics
import numpy as np
import tensorflow as tf 
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping


df = pd.read_csv("/content/drive/My Drive/StudentsPerformance.csv")

# Generate dummies.
df = pd.concat([df,pd.get_dummies(df["gender"],prefix="gender")],axis=1)
df.drop("gender",axis=1, inplace=True)
df = pd.concat([df,pd.get_dummies(df["race/ethnicity"],prefix="race/ethnicity")],axis=1)
df.drop("race/ethnicity",axis=1, inplace=True)
df = pd.concat([df,pd.get_dummies(df["parental level of education"],prefix="parental level of education")],axis=1)
df.drop("parental level of education",axis=1, inplace=True)
df = pd.concat([df,pd.get_dummies(df["lunch"],prefix="lunch")],axis=1)
df.drop("lunch",axis=1, inplace=True)
df = pd.concat([df,pd.get_dummies(df["test preparation course"],prefix="test preparation course")],axis=1)
df.drop("test preparation course",axis=1, inplace=True)

# Create a total score
df["total score"] = df["math score"]+df["reading score"]+df["writing score"]
x_columns = df.columns.drop("math score").drop("reading score").drop("writing score").drop("total score")
x = df[x_columns].values
y = df["total score"].values

kf = KFold(5,shuffle=True,random_state=97)

oos_y = []
oos_pred = []

fold = 0
for train,test in kf.split(x):
  fold+=1
  print(f"Fold #{fold}")
  x_train = x[train]
  y_train = y[train]
  x_test = x[test]
  y_test = y[test]

  #Build the neural network
  model = Sequential()
  model.add(Dense(25,input_dim=x.shape[1],activation="relu")) # hidden 1 
  model.add(Dense(10, activation="relu")) # hidden 1 
  model.add(Dense(1))
  model.compile(loss="mean_squared_error",optimizer="adam")
  monitor = EarlyStopping(monitor="val_loss",min_delta=1e-3,patience=30,verbose=1,mode="auto",\
    restore_best_weights=True)
  model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks = [monitor], verbose=2,epochs=2000)
  pred = model.predict(x_test)
  #RMSE
  oos_y.append(y_test)
  oos_pred.append(pred)

  RMSE_score = np.sqrt(metrics.mean_squared_error(pred,y_test))
  print("Fold Score:{}".format(RMSE_score))
#Calculate the error
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
score = np.sqrt(metrics.mean_squared_error(oos_pred,oos_y))
print(f"Final RMSE:{score}")

Fold #1
Epoch 1/2000
25/25 - 0s - loss: 42632.9688 - val_loss: 44950.1641
Epoch 2/2000
25/25 - 0s - loss: 42450.2539 - val_loss: 44703.1133
Epoch 3/2000
25/25 - 0s - loss: 42122.1484 - val_loss: 44246.8750
Epoch 4/2000
25/25 - 0s - loss: 41534.2461 - val_loss: 43440.6133
Epoch 5/2000
25/25 - 0s - loss: 40500.8828 - val_loss: 42081.8555
Epoch 6/2000
25/25 - 0s - loss: 38856.4648 - val_loss: 40012.9141
Epoch 7/2000
25/25 - 0s - loss: 36456.8398 - val_loss: 37094.5469
Epoch 8/2000
25/25 - 0s - loss: 33204.5508 - val_loss: 33277.5742
Epoch 9/2000
25/25 - 0s - loss: 29143.7070 - val_loss: 28756.6152
Epoch 10/2000
25/25 - 0s - loss: 24547.7148 - val_loss: 23804.4355
Epoch 11/2000
25/25 - 0s - loss: 19724.8340 - val_loss: 18772.5547
Epoch 12/2000
25/25 - 0s - loss: 15017.7451 - val_loss: 14071.9434
Epoch 13/2000
25/25 - 0s - loss: 10827.3096 - val_loss: 10015.5049
Epoch 14/2000
25/25 - 0s - loss: 7412.6538 - val_loss: 6847.8657
Epoch 15/2000
25/25 - 0s - loss: 4929.0576 - val_loss: 4635.0273


In [0]:
# We successfully lowered the RMSE a bit.
# However, when we are using the Kfold, it also means we used all the dataset.